#Instalation and Imports

This notebook is an exercise and exploration about one of my most belove interest: the astronomy. And I'm posting my notebook here, and also on github, to share my passion and results with more people that also love science and Data Science.

Please, if you found any errors (code, grammar, astronomical or mathematical erros) here, report me and I'll try to improve this notebook the best that i can (i'm not a professional, i just love astronomy and data science). Thank you, and have a nice jouney through the cosmos

In [ ]:
!pip install kneed

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from kneed import KneeLocator

import scipy
import scipy.cluster.hierarchy as shc
from scipy.stats import shapiro, boxcox, t, mannwhitneyu
from statistics import stdev
from math import sqrt
from statsmodels.graphics.gofplots import qqplot

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

sns.set()

#The dataset

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
combo = pd.read_csv('../input/combo17-galaxy-dataset/COMBO17.csv')

In [ ]:
combo.head()

## Changing feature names to better understanding

In [ ]:
combo = combo.rename(columns={ 'UjMAG': 'UVmag', 'BjMAG': 'Bluemag', 'VjMAG': 'Opticmag',
        'W420FE': 'lenwave1', 'W462FE': 'lenwave2', 'W485FD': 'lenwave3',
       'W518FE': 'lenwave4', 'W571FS': 'lenwave5', 'W604FE': 'lenwave6',
       'W646FD': 'lenwave7', 'W696FE': 'lenwave8','W753FE': 'lenwave9',
        'W815FS': 'lenwave10',  'W856FD': 'lenwave11', 'W914FD': 'lenwave12',
        'W914FE': 'lenwave13', 'UFS	': 'UVfilter', 'BFS': 'BLUEfilter', 'VFD': 'OPTICfilter', 'RFS': 'REDfilter', 'IFD': 'InfraREdfilter'})

In [ ]:
combo.head(2)

##Exploratory Data Analysis 

In [ ]:
combo.shape

In [ ]:
for c in combo.isnull().sum() / combo.shape[0]:
  print(c)

In [ ]:
combo = combo.dropna()
combo.shape

In [ ]:
combo.info()

### Dropping not very usefull features to us

In [ ]:
error_features = []
for feature in combo.columns:
  if feature[0] == 'e':
    error_features.append(feature)

In [ ]:
combo = combo.drop(error_features, axis=1)

In [ ]:
combo.columns

### Describe and Relaation inside data  

In [ ]:
combo.describe()

In [ ]:
plt.figure(figsize=(12,9))
sns.heatmap(combo.corr())

## Understanding the three main variables

Working with three main features

- Rmag (Red Magnetude - Bright)

- ApDRmag (Size)

- Mcz (redshift - Distance)

This features can e analysed as main properties of the galaxies, so let's see how they are related to eac other, and try to find some interesting pattern.

In [ ]:
#defining a function of find the Person Correlation and the R square of each combination of features
def find_corr_galaxy(x=0, y=0):
  corr_matrix = np.corrcoef(x, y)
  pearson_corr_galaxy = corr_matrix[0,1]
  r2_galaxy = pearson_corr_galaxy ** 2
  return pearson_corr_galaxy, r2_galaxy

In [ ]:
#as we can see, there is a good correlation between this two features, and also a strong R2
#that means 47% of the variance in one features can be explnaed by the variance  in the other one
pearson_corr, r2 = find_corr_galaxy(x=combo.Rmag, y=combo.ApDRmag)
print(f'Pearson Correlation Bright vs Size: {pearson_corr:.2f}')
print(f'R2: {r2:.2f}')

In [ ]:
#ploting this correlation helps us to a better understanding of the situation 
plt.figure(figsize=(15,10))
sns.scatterplot(x='Rmag', y='ApDRmag', data=combo)

In [ ]:
#here its clearly that the correlation is not that strong
#only 4% of the variance in the size can be explaned by the distance of the galaxy from us
pearson_corr, r2 = find_corr_galaxy(x=combo.Mcz, y=combo.ApDRmag)
print(f'Pearson Correlation Distance vs Size: {pearson_corr:.2f}')
print(f'R2: {r2:.2f}')

In [ ]:
#ploting this correlation its clear that the correlation is weak
#but still it looks like a tendency
plt.figure(figsize=(15,10))
sns.scatterplot(x=combo.Mcz,y=combo.ApDRmag)

In [ ]:
#and we have a reasonable between color and bright 
pearson_corr, r2 = find_corr_galaxy(x=combo.Mcz, y=combo.Rmag)
print(f'Pearson Correlation Distance vs Bright {pearson_corr:.2f}')
print(f'R2: {r2:.2f}')

In [ ]:
plt.figure(figsize=(9,6))
sns.scatterplot(x=combo.Mcz,y=combo.Rmag)

Relationship Bright x lenght wave

- A interesting point to be known is that more bright the galaxy is more intense will be the emission of eletromagnetic waves, and we can take a look if this happen hear ploting the Magnitude of the galaxy but the intensity of fotons that are recived here for each length wave.

In [ ]:
#deffining a function to plot all the length waves by magnitude
def plot_lenwave(column_name): 
  plt.figure(figsize=(15,10))

  sns.lineplot(x = combo[column_name], y = combo['REDfilter'], palette = 'red', label = 'RED-FILTER' )
  sns.lineplot(x = combo[column_name], y = combo['BLUEfilter'], palette = 'blue', label = 'BLUE-FILTER')
  sns.lineplot(x = combo[column_name], y = combo['OPTICfilter'], palette = 'green', label = 'GREEN-FILTER')
  sns.lineplot(x = combo[column_name], y = combo['UFS'], palette = 'Purple', label = 'ULTRAVIOLET-FILTER')
  sns.lineplot(x= combo[column_name], y = combo['InfraREdfilter'], palette = 'yellow', label = 'INFRARED-FILTER')
  plt.ylabel('Wave length Intensity')
  plt.legend()
plot_lenwave(column_name = 'Rmag')

The plot above confirms our thought, the less the magnitude, the more intense is the emition of fotons (waves) 

Here there is a pick in the of emission between the galaxies with -2 and 0. Comparing with the previous plot we can infer that galaxies between this range are the galaxies with low magnitude, and by consequence, more blue

In [ ]:
plot_lenwave(column_name='ApDRmag')

Now its clear that the galaxies near to us are the ones with most intense emission.  

In [ ]:
plot_lenwave(column_name='Mcz')

###What do we get from these plots?

- So, based in this plots we can already now that we have a clear tendency to have more bright and median size galaxies near from us. And as long you keep going more and more away from our galaxy you will find less bright galaxies and also bigger.

###Creating a new feature:

- when you are studing astronomy its preatty comon that you have to understand what is a color index. A color index is the measurement of how much ligth he galaxy provides in an especifc range in the eletromagnetic spectrum. That's why to know the color index of any object we have to subtract the magnitude of an object in an especifc magnitude in the eletromagnetic spectrum from the magnitude in another point of the eletroc=magnetic spectrum.

- The most common color index that we have is the BV-index, but the UB-index its also used a lot.

In [ ]:
#creating columns for color-index
combo['BV_color_index'] = combo.Bluemag - combo.Opticmag
combo['UB_color_index'] = combo.UVmag - combo.Bluemag

In [ ]:
combo.head(1)

In this point we have reached one of the most amazing parts of the data visualization with astronomy. Ploting the Distance of the galaxies from us, and the UB-color_index, the espectrum between the Ultra violet and the Optic Blue we can find a really interesting patter. Each point is a galaxy, relaed to the distance and the UB-color_index. And as long you go by the x-axis (the distance) you can find clear vertical bands of points. Those band are cluster os galaxies! Together in the same point of space they share the same distance, because due to the gravitacional force they are grouped together and start to form groups and clsuter, just like our Local Group and the Virgo Supercluster. And we can clearly see the results of those incredible gravitation force IN FRONT OF OUOR EYES. If thats not beautiful i don't know what else is.

In [ ]:
#to be able to find this plot we need to cut some outliers that can make some noise nad create trouble
#thats why we use .loc 
plt.figure(figsize=(15,10))
sns.scatterplot(x=combo.Mcz.loc[(combo.UB_color_index>-1.5)&(combo.UB_color_index<1)], y=combo.UB_color_index, size=0.3)

In [ ]:
#we can find the same result here with another color index.
plt.figure(figsize=(15,10))
sns.scatterplot(x=combo.Mcz.loc[(combo.BV_color_index>-1.5)&(combo.BV_color_index<1)], y=combo.BV_color_index, size=0.3)

#Cluster Analysis

Know, let's ay we want to group all our galaxies into groups, analyse if there is a pattern going on here. But we want to do that with some relevant features for the galaxies, thing taht metter the most. For a galaxy clusterization i choosed three main features: 

- The DISTANCE of the galaxy from us (MCZ feature correspond to it, its the RedShift of the galaxy. The greater the RedShift the Greater the distance)

- The MAGNITUDE IN RED of the galaxy (Rmag feature. The greater the magnitude more red a galaxy is, and by consequence less bright it is. Low magnitude usually means blue and brighter colors, and bigmagnitudes means red and less bright galaxies)

- The SIZE of the galaxy (ApDRmag is due to this. This is relative to the total and apperent magnitude of the galaxy)

In [ ]:
#separeting this features 
combo_cluster = combo.iloc[:,[1,2,4]]

##Dealing with outliers:

Here we have some specificities of our dataset. Galaxies have a huge distributions of colors, shapes, and distances. To be able to do our cluster and also have a representative of the variance in the characteristics of those galaxies we will cut only the EXTREMY outliers, and the ones who will make some trouble to us in the future. (I test tons of possibilities of cuts for those outliers, the best trade of that i founded are seted bellow.)

Disclaimer: usually cut outliers helps a lot distance based algorithms, just like K-means, for example, but that's not an absolute true. usually you have to try a lot of possibilities and find the best fit to your data specifities.

In [ ]:
#boxplot to visualize the Magnitude of our galaxies
#they are great to look for outliers
sns.boxenplot(combo_cluster.Rmag)

In [ ]:
#histogramm of the same data above
#it allows us for look to outliers and have some notion of the sahpe of our data
#a right-skewed shape in this case
combo_cluster.Rmag.hist()

In [ ]:
#let's cut the some bad outliers here
combo_cluster = combo_cluster.loc[combo_cluster['Rmag'] >18]

In [ ]:
#boxplot for Maganitude after the cut 
#it seems better, and didin't affected to much the galaxies characteristics
sns.boxenplot(combo_cluster.Rmag)

In [ ]:
#boxplot for galaxy Size
#here we have some really bad outliers, we need to be a little more agressive with them this time
sns.boxplot(combo_cluster.ApDRmag)

In [ ]:
#histogram from the same data abpve
combo_cluster.ApDRmag.hist()

In [ ]:
#cuting the outliers, but still alows to small and giant galaxies appers in the dataset
combo_cluster = combo_cluster.loc[combo_cluster['ApDRmag'] >-2]

In [ ]:
#boxplot of the size after the cut
sns.boxplot(combo_cluster.ApDRmag)

In [ ]:
#the distnce of the galaxies from us do not have some baroutliers
sns.boxplot(combo_cluster.Mcz)

In [ ]:
#histogram representing a bimodal data (if you want to see this better, increase the number of the bins) 
combo_cluster.Mcz.hist()

In [ ]:
#after some search i founded a good point to discart the mot distant galaxies.
combo_cluster = combo_cluster.loc[combo_cluster['Mcz']<1.2]

In [ ]:
combo_cluster['Mcz'].hist()

In [ ]:
#taking a look in outr data right now
combo_cluster.head()

In [ ]:
#there is no missing that here, but jjust for safety:
combo_cluster = combo_cluster.dropna()

## The Algorithms

Here we going to create three different versions of our dataset. One will be the pure dataset, and the other twos will pass for some normalization. Usually the normalization the algorithms based in distances between the poits because it set the same "weigth" to the features. It do not allow tat features with huge scales (maginitude -Rmag- for example, who goes from 18 to 26) affect the model much more than features with tiny scales (just like Mcz, each goes from 0 to 1.2)

### Scaling the Dataset

In [ ]:
scaler = StandardScaler()
min = MinMaxScaler()
combo_cluster_scaler = scaler.fit_transform(combo_cluster)
combo_cluster_min = min.fit_transform(combo_cluster)
combo_cluster_scaler

Now we are going to start our clusterization. We'll test three main algorithms: K-MEANS, AGGLOMERTAIVE CLUSTERING and DBSCAN.

###AGGLOMERATIVE CLUSTERING: 

A clustering based in the idea of starts a cluster with just one point, and from this point, start the aggregation of this points, creating a dendogram based in the distance from each point. The idea is creat small cluster that starts to agregate and create big clusters.

- The distance between the points can be calculated based in the minimun distance possible etween the clusters, the maxium, the average and so on. here we are going to try soome of them with he euclidean distance.

- Our metric for cluster anlysis will be the Silhouette score. The silhouette score measures the average distance between the intra-cluster points and the inter-clusters. Small intra-cluster distance are btter, the points inside the cluster have good cohesion, and big inter-cluster distance are very good, because it means that the clusters are very far fom each other and have a good delimitation. In the Silhouette score greater is better.

In [ ]:
def agglomerative_cluster(dataset):
  silhouette_coefficient = []   #list to silhouette score
  for linkage in ['ward', 'complete', 'average', 'single']:   # the distance measuremtn will be based in
    for number in range(2,11):
      agglomerative = AgglomerativeClustering(n_clusters= number, affinity='euclidean', linkage = linkage) #agglomerative model
      agglomerative.fit(dataset)    #fitting the model
      score = silhouette_score(dataset, agglomerative.labels_)
      silhouette_coefficient.append(score)

  #as the list is not separed by the linkage type, we can slice the list into peaces that we know 
  #that represent each type of linkage. take the linkage that generate the best silhouette score to analyse it.    
  print(f'ward: {max(silhouette_coefficient[:9])}, {len(silhouette_coefficient[:9])}')
  print(f'maximun: {max(silhouette_coefficient[9:18])}, {len(silhouette_coefficient[9:18])}')
  print(f'average: {max(silhouette_coefficient[18:27])}, {len(silhouette_coefficient[18:27])}')
  print(f'minimun: {max(silhouette_coefficient[27:36])}, {len(silhouette_coefficient[27:36])}')

agglomerative_cluster(dataset=combo_cluster)

In [ ]:
agglomerative_cluster(dataset=combo_cluster_min)

In [ ]:
agglomerative_cluster(dataset=combo_cluster_scaler)

The Agglomertaive cluster sets a random point to starts the cluster, That results in different cluster each times you run the algorithm. So the silhouette score will not be the same when yu run it. But here our best score is around 0.61 (best value possible is 1, the worst is 0)

In [ ]:
silhouette_coefficient = []
for linkage in ['ward', 'complete', 'average', 'single']:
  for number in range(2,11):
    agglomerative = AgglomerativeClustering(n_clusters= number, affinity='euclidean', linkage = linkage)
    agglomerative.fit(combo_cluster_scaler)
    score = silhouette_score(combo_cluster_scaler, agglomerative.labels_)
    silhouette_coefficient.append(score)
      
print(f'ward: {max(silhouette_coefficient[:9])}, length: {len(silhouette_coefficient[:9])}')
print(f'maximun: {max(silhouette_coefficient[9:18])}, length: {len(silhouette_coefficient[9:18])}')
print(f'average: {max(silhouette_coefficient[18:27])}, length: {len(silhouette_coefficient[18:27])}')
print(f'minimun: {max(silhouette_coefficient[27:36])}, length: {len(silhouette_coefficient[27:36])}')

In [ ]:
#Let's plot the silhouette coefficient by number of clusters for the best linkage
plt.style.use('fivethirtyeight')
plt.plot(range(2,11), silhouette_coefficient[18:27])
plt.xticks(range(2,11))
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.show()

Here we can see one of the prolems that we must deal in this case. Our best agglomerative clustering is good beacuse in creates a cluster who involves almost all the points in the dataset, being just a few points out of that cluster. This means that our cluster its not relevant for our analysis. You can see it graphically in the dendogram bellow. Just one color means there is just one cluster.

In [ ]:
plt.figure(figsize=(15, 10))
plt.title('Galaxy Dendograms')
dend = shc.dendrogram(shc.linkage(combo_cluster_scaler, method='average'))

###K-MEANS:

In the K-mEANS algorithm you MUST chose the number of clusters that you want to set. This number will create the same nuber of centroids. This centroids will be used as the center of the cluster. In each iteration of the algorithm the centroid will migrate to the best possible place, where the average of the distance of this centroid to the nerest points are the smallest as possible. 

- Thats why we need to set a Number of iterations, in this case: 500
- need to set a number of the cluster (centroids) to be created
- the init parameter can be used as 'random', so the centroids will be seted randomly is the n-dimensional space (here: 3D space). But when you set the k-means++ parametrs the centroids will be seted with some distance between those centroids, helping the algorithm to converge. 

In [ ]:
# as before, try different combinations of cluster hyperparameters, and saving the sum of square error 
def sse_kmeans_cluster(dataset):
  sse = []
  for k in range(1,11):
    kmeans = KMeans(n_clusters = k, init='k-means++', n_init=50, max_iter=500, random_state=42 )
    kmeans.fit(dataset)
    sse.append(kmeans.inertia_)
  print(sse)
  return sse
sse = sse_kmeans_cluster(dataset=combo_cluster)

In [ ]:
# here we plot the sum of square error that each combination of clusters
#as we can see, around the two and 3 cluster we have a good trade of between low error and a goos number of clusters
plt.style.use('fivethirtyeight')
plt.plot(range(1,11), sse)
plt.xticks(range(1,11))
plt.xlabel('Number of Clusters')
plt.ylabel('Error')
plt.show()

Here we are going to use a k new method: the elbow method. Using the derivative of the function, and where it have the greatest value will be you elbow, be best trade of for you.

In [ ]:
knee = KneeLocator(
    range(1,11), sse, curve = 'convex', direction = 'decreasing'
)
print(f'The flexure points: {knee.elbow}')

In [ ]:
#now using the silhouette coefficient
#first with the combo_cluster dataset
def silhouette_kmeans_cluster(dataset):
  silhouette_coefficient = []
  for k in range(2,11):
    kmeans = KMeans(n_clusters = k, init='k-means++', n_init=30, max_iter=500, random_state=42 )
    kmeans.fit(dataset)
    score = silhouette_score(dataset, kmeans.labels_)
    silhouette_coefficient.append(score)
  return silhouette_coefficient
silhouette_kmeans_cluster(dataset=combo_cluster)

In [ ]:
# now with combo_cluster_min
silhouette_kmeans_cluster(dataset=combo_cluster_min)

In [ ]:
#ploting with combo_cluster_min
#and here we have the ebst k-means possible
plt.style.use('fivethirtyeight')
plt.plot(range(2,11), silhouette_coefficient[:9])
plt.xticks(range(2,11))
plt.xlabel('Number of clusters')
plt.ylabel('coeficiente de silhouette')
plt.show()

In [ ]:
#with combo_cluster_scaler
silhouette_kmeans_cluster(dataset=combo_cluster_scaler)

###DBSCAN:

The Density Based Spacial Clustering for Application with Noise) we have a different approach, Here we need three types of points:

- Core points: this points will be used as "centers" for N other points (choosen by you) inside a specific radius, where all the other points around will be used as border points

- Border points: this points is every single point wich do not have N points around them, like the core points

- Outlier is a point wich are not reachable for core points , and not a border point

This means, it uses distance, but also deal with non spherical clusters, and with random shapes. And maybe it will be usefull for our problem.

In [ ]:
def dbscan_cluster(dataset):
  silhouette_coefficient=[]
  #setting radius for core points
  for radius in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    #setting number of points inside the radius
    for n_samples in [2,3,4,5,6,7,8,9,10]:
      dbscan = DBSCAN(eps=radius,min_samples = n_samples)
      dbscan.fit(dataset)
      score = silhouette_score(dataset, dbscan.labels_)
      silhouette_coefficient.append(score)
      
  print(f'radius-0.1: {max(silhouette_coefficient[:9])}, {len(silhouette_coefficient[:9])}')
  print(f'radius-0.2: {max(silhouette_coefficient[9:18])}, {len(silhouette_coefficient[9:18])}')
  print(f'radius-0.3: {max(silhouette_coefficient[18:27])}, {len(silhouette_coefficient[18:27])}')
  print(f'radius-0.4: {max(silhouette_coefficient[27:36])}, {len(silhouette_coefficient[27:36])}')
  print(f'radius-0.5: {max(silhouette_coefficient[36:45])}, {len(silhouette_coefficient[36:45])}')
  print(f'radius-0.6: {max(silhouette_coefficient[45:54])}, {len(silhouette_coefficient[45:54])}')
  print(f'radius-0.7: {max(silhouette_coefficient[54:63])}, {len(silhouette_coefficient[54:63])}')
  print(f'radius-0.8: {max(silhouette_coefficient[63:72])}, {len(silhouette_coefficient[63:72])}')
  print(f'radius-0.9: {max(silhouette_coefficient[72:81])}, {len(silhouette_coefficient[72:81])}')


In [ ]:
# using combo_cluster
dbscan_cluster(dataset=combo_cluster)

In [ ]:
#the combo_cluster_min creates an unique cluster, thats not usefull for us
#dbscan_cluster(dataset=combo_cluster_min)

In [ ]:
#using the combo_cluster_scaler
dbscan_cluster(dataset=combo_cluster_scaler)

In [ ]:
#using the best dbscan 
dbscan_silhouette_coefficient=[]
for n_samples in [2,3,4,5,6,7,8,9,10]:
  dbscan = DBSCAN(eps=0.7,min_samples = n_samples)
  dbscan.fit(combo_cluster)
  score = silhouette_score(combo_cluster, dbscan.labels_)
  dbscan_silhouette_coefficient.append(score)

In [ ]:
#using radius of 0.7 and searching for number of samples inside this radus, in this case: 7 
plt.style.use('fivethirtyeight')
plt.plot(range(2,11), dbscan_silhouette_coefficient)
plt.xticks(range(2,11))
plt.xlabel('number of Samples')
plt.ylabel('Silhouette Coefficient')
plt.show()

###Best algorithms and hyperparameters

To a better visualization of our tools i will display our bests options for each algorithm, and then we set their classification for each point in the combo-cluster dataset

In [ ]:
#train agglomerative cluster with the best hyperparameters
agglomerative_cluster = AgglomerativeClustering(n_clusters = 2, linkage='average', affinity='euclidean')
agglomerative_cluster.fit_predict(combo_cluster_scaler)

#append the labels to every galaxy 
combo_cluster['hierarchical_label'] = agglomerative_cluster.labels_

In [ ]:
# train kmeans cluster with the best hyperparameters
kmeans_model = KMeans(n_clusters = 2, init='k-means++', n_init=50, max_iter=500, random_state=42 )
final_kmeans = kmeans_model.fit(combo_cluster_min)
kmeans_label = final_kmeans.labels_

#append the labels to every galaxy
combo_cluster['kmeans_label'] = kmeans_label

In [ ]:
#train DBSCAN cluster with the best hyperparameters
dbscan_model = DBSCAN(eps=0.7, min_samples=7)
best_dbscan = dbscan_model.fit(combo_cluster)
dbscan_labels = best_dbscan.labels_

#append label
combo_cluster['dbscan_label'] = dbscan_labels 

In [ ]:
# taking a look in the dataset 
combo_cluster.head(3)

## Understanding the Results

In [ ]:
# here we can see our greatest problem with the agglomertaive cluster
# almost every point were classified as the same cluster, that's not usefull 
# REMEMBER: silhouette_coefficient = 0.57
combo_cluster['hierarchical_label'].value_counts()

In [ ]:
#here we hhave a better distribution, with two clusters created
#REMEMBER: silhouette_coefficient = 0.49
combo_cluster['kmeans_label'].value_counts()

In [ ]:
# here we have 3 clsuter 9notice: the third cluster (-1) was created based in the extreme values of the label 0 from kmeans 
# REMEMBER: silhouette_coefficient = 0.66 (our best value) 
combo_cluster['dbscan_label'].value_counts()

### 3D Plots

Now we can see in a 3D plot how the clusters were asign to the galaxies

- Firts when can see that in the AGGLOMERATIVE cluster all the points have the same color (red), that means they are part of the same cluster, there is no difference between them, and we can see  that the points are not equal, or part of the sam estructure. So that won't be usefull for us. 

- That was problably caused by our outliers here, but i still prefer keep these points to be plausible with he reality.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

cores = {0:'red', 1:'blue'}

fig = plt.figure(figsize=(21, 15))
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(combo_cluster.Rmag, combo_cluster.ApDRmag, combo_cluster.Mcz,
           c= combo_cluster.hierarchical_label.apply(lambda x: cores[x]))
ax.set_title('Hierarchical 3D DISTRIBUTION')
ax.set_xlabel('MAGNITUDE(Rmag)')
ax.set_ylabel('TAMANHO (ApDRmag)')
ax.set_zlabel('DISTANCIA (Mcz)')

Using the K-MEANS to display our points we can see a better clusterization, where we have two groups of galaxies, a little bit overlaped, but far better than the agglomertaive algorithm. That overlaping usually is due to the similarity between some galaxies, that probably happen because some galaxies are in transition from one group to another, or just anomalous galaxies. 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

cores = {1:'red', 0:'blue'}

fig = plt.figure(figsize=(21, 15))
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(combo_cluster.Rmag, combo_cluster.ApDRmag, combo_cluster.Mcz,
           c= combo_cluster.kmeans_label.apply(lambda x: cores[x]))
ax.set_title('KMeans 3D DISTRIBUTION')
ax.set_xlabel('MAGNITUDE(Rmag)')
ax.set_ylabel('TAMANHO (ApDRmag)')
ax.set_zlabel('DISTANCIA (Mcz)')

Finally we have our DBSCAN classification, and as we can see there are three cluster now, two very similar to the K-means cluster, and one more with some extremy vaues related with the maginitude and the size of the galaxy. One consequence of the good clusterization is that here we have less overlaping than before, that could the know by the better silhouette coeficcient.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

cores = {0:'red', 1:'blue', -1:'green', 2:'yellow'}

fig = plt.figure(figsize=(21, 15))
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(combo_cluster.Rmag, combo_cluster.ApDRmag, combo_cluster.Mcz,
           c= combo_cluster.dbscan_label.apply(lambda x: cores[x]))
ax.set_title('DBSCAN 3D DISTRIBUTION')
ax.set_xlabel('MAGNITUDE(Rmag)')
ax.set_ylabel('TAMANHO (ApDRmag)')
ax.set_zlabel('DISTANCIA (Mcz)')

Taking a look by other positions. Lokking by the side of the plot we can see a non spherical cluster, and thats why the DBSCAN got better results than the other algorithms. The blue uster seems to be nearest to the Earth, and in general have lower magnitudes.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

cores = {0:'red', 1:'blue', -1:'green'}

fig = plt.figure(figsize=(21, 15))
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(combo_cluster.Rmag, combo_cluster.ApDRmag, combo_cluster.Mcz,
           c= combo_cluster.dbscan_label.apply(lambda x: cores[x]))
ax.set_title('DBSCAN 3D DISTRIBUTION')
ax.set_xlabel('MAGNITUDE(Rmag)')
ax.set_ylabel('TAMANHO (ApDRmag)')
ax.set_zlabel('DISTANCIA (Mcz)')
ax.view_init(0,90)

A view from above shows us the relationship between size and magnitude. The more the size ncreases more red is the galaxy. Thats not a rule, and there is some overlaping there, but still possible to understand that. (Remeber, the greater the magnitude, more red and less brigth the galaxy is (and usually bigger).

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

cores = {0:'red', 1:'blue', -1:'green'}

fig = plt.figure(figsize=(21, 15))
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(combo_cluster.Rmag, combo_cluster.ApDRmag, combo_cluster.Mcz,
           c= combo_cluster.dbscan_label.apply(lambda x: cores[x]))
ax.set_title('DBSCAN 3D DISTRIBUTION')
ax.set_xlabel('MAGNITUDE(Rmag)')
ax.set_ylabel('TAMANHO (ApDRmag)')
ax.set_zlabel('DISTANCIA (Mcz)')
ax.view_init(90,270)

## the means of the Cluster

Now we can group by the dataset by the labels that we created with the clustering algorithms, finding the mean of each feature and knowing the average value of each feature that we want.

In [ ]:
combo_cluster.groupby('kmeans_label').mean()

In [ ]:
combo_cluster.groupby('dbscan_label').mean()

# Statistical Analysis

As we can se, the DBSCAN algorithm did a better job than the other ones. In fact it did a silhouette score of 0.64. Not a bad score. This algorithm returned three cluster. Where greater Rmag means more red galaxies,  more positive ApDRmag means bigger galaxies and higher Mcz is related to far galaxies, and here is their analysis:
  

- Label 0: Here we have a clear Red Giant galaxies. Those kind os struture usualy are bigger tahn the other, because they were formed by the merge of other galaxies, like two blue galaxies or small red galaxies.

- Label 1: A variety of blues galaxies. There are two types of galaxies, barred spiral galaxies and non-barred spiral galaxies. We can't tell with those data what kind each galaxy is, but with The Magnitude in Red (Rmag) we can clearly say that are Blue median size galaxies in avarege.

- Label -1:There just a few samples of that type here, so its hard to be sure os what we're seeing. but it seems to be a very intense and dwarf blue galaxies. With a very low Rmag and small size comparing with the two other samples.

##EVALUATING SIZE

###Hypothesis Test with Two sample T-test

With those datas into our hands we can start to try to figure some queestions that could come to your mind. A good thought is: That difference in size between these two samples are true ? Can we say these two populations are really different by size? 

To to that we can just to a simple Two Samples T-test. In that case we can treat this the groups of galaxies (Label 0 and label 1) as two samples of all the galaxies that we could have. Label 0 will be called as the Red population and Label 1 will be called as Blue population. So, with these the samples in our hands we are capable to look for the difference in the mean size of these two groups of galaxies. To do that we're going to create a Hypotesis Test, wich is a test to validate if two groups are similar enough or not. So we create a Null Hypothesis, wich is the idea of the mean size of these two samples are  similar enough to considere then as part of the same population galaxies size. An we create an Alternative Hypothesis, that tells us teh mean sie os those groups are so different that we could teel the are part of two distinquisy populations. O let's put our hand on the hypothesis test:

- Hypothesis Tes
  - H0: mean size Red sample = mean size Blue sample
  - H1: mean size Red sample != mean size Blue sample

- Alpha level:
  -  0.05: this number set an acceptable level of certainty that we need to have to retain or reject the null hypothesis. This means we need to have less than 5% of probability to take a t-test result by chance and not due to a true characteristics of the mean



###EVALUATING THE NORMALITY OF OUR DATA: ( we have three aprouches

  - Histogram: the easier and less acurated way to analyse a distribution. Its a good start but ever the point of a last decision.

  - QQplot: acording to some discutions s the most acurated way to vizualize the normalite of the data, especialy for tons of data, just like our case. here we plot all the points of our distributions against the expected quartils of a normal distribution. If your point follow the normal line of those quartil you can considere your data as a normal or like-normal curve.

  - Normality test (Shapiro Wilk test): a good statistical test, where H0 is the hypothesis of normality of your data and H1 is the non-normality of the data. If p > 0.05 you have a normal curve, else, your curve is not normal. Acording to my researchs this test is affected for just a few differences in your data from the normal curve when your dataset is hugh. So we'll be not using those test as a last word for normality. 

In [ ]:
#taking a look in all ApDRmag points
#it looks like normal, but with ligth right-skewed data
combo_cluster['ApDRmag'].hist(bins=20)

In [ ]:
qqplot(combo_cluster['ApDRmag'], line='s')
plt.show()

In [ ]:
#as we can se, despite of the normal like curve, 
#the shapiro wilk test tells us our data is really far from the normality [p(0.05)>>>>>0.2085440954504141e-21]
stats, p = shapiro(combo_cluster['ApDRmag'])
print(stats, p)

In [ ]:
#here we'll cut some outliers, mainly the smallest values, that is due to Label -1 (dwarf galaxies), so we can cut them
size = combo_cluster['ApDRmag'].loc[(combo_cluster['ApDRmag'] >-1.08)&(combo_cluster['ApDRmag']<1.3)] 
size.hist(bins = 30)


In [ ]:
#here we can clearly see that the data is much more normal-like data
qqplot(size, line='s')
plt.show()

In [ ]:
stats, p = shapiro(size)
print(f'{stats}, {p:.10f}')

Now we can considure our data as a normal like data. So lets start to separete our data and see how their distribution looks like.

Firts of all we need to separate our data in two groups. One group will cointain the size of Label 0 group, wich we're going to call "red_sample", and the other will contain the size from Label 1 group, the "blue_sample. 

In [ ]:
red_sample = combo_cluster['ApDRmag'].loc[(combo_cluster['ApDRmag'] >-1.08)&(combo_cluster['ApDRmag']<1.3)&(combo_cluster['dbscan_label']==0)]
blue_sample = combo_cluster['ApDRmag'].loc[(combo_cluster['ApDRmag'] >-1.08)&(combo_cluster['ApDRmag']<1.3)&(combo_cluster['dbscan_label']==1)]

RED SAMPLE

Does the red sample follow the normality?

In [ ]:
red_sample.hist(bins=20)

In [ ]:
qqplot(red_sample, line='s')
plt.show()

BLUE SAMPLE

Does blue_sample follow the normality?

In [ ]:
blue_sample.hist(bins=20)

In [ ]:
qqplot(blue_sample, line='s')
plt.show()

###The T-test

Now we are able to start our TWO SAMPLE T-TEST for INDEPENDENT SAMPLES

To be able to do this test you need to know how far the the means are from each other in standard deviation units. To find that you can use the formula:
  - t-statistc = difference of the sample means /standard error of the difference between the means
  - and the standard error of the differences between the means is:
    - The square of standard error from the first sample / number of samples inside of sample 1 +   square of standard error from the second sample / number of samples inside of sample 2

Here we are going to create a step-by-step two sample t-test for independent samples. That's good if you want to understand better how do we get the final result. After that we can compare it with a scipy function that returns a similar function

In [ ]:
#let's deffine a function to find the t-statistic
def t_statistic(): 
  #set an alpha value 
  alpha = 0.05
  
  #first lets find the red sample mean and the blue sample mean, then find their differencw
  bsm = blue_sample.mean() #blue sample mean
  rsm = red_sample.mean() #red sample mean
  dsm = rsm - bsm #difference of sample means 

  #now we need to find the square root of the differences between the menas
  #find the standard deviation of the samples. I'll be using Benso's correction fot samples
  std_rs = stdev( red_sample) #for population use pstdev()
  std_bs = stdev(blue_sample)

  #find number of samples in each group
  n_rs = len(red_sample)
  n_bs = len(blue_sample)

  #calculate standard error of the difference between the means
  dbm = sqrt(((std_rs**2)/n_rs) + ((std_bs**2)/n_bs))

  #t-statistic
  t = dsm / dbm 

  # find the degrees of freedom
  df = n_rs+n_bs - 2

  #find the critical value using the  percent point function (ppf)
  crit_val =  scipy.stats.t.ppf(1.0 -alpha, df)

  #find p-value with cumulative distribution function (cdf)
  # *2 because we dont care if the the blue mean is bigger or smaller than the red mean (two-tailed t-test)
  p = (1 - scipy.stats.t.cdf(abs(t), df)) * 2 



  return t, df, crit_val, p
t_stats, df, cv, p = t_statistic() 
print(f'T-statistc: {t_stats}')
print(f'Degrees of freedom: {df}')
print(f'T-critical value: {cv}')
print(f'P-value: {p}')

Here we will evaluate our hypothesis:

- With an alpha level of 0.05, and a P-value so low that our function can't reach the true value we can take the following thought:
  - P < alpha: WE REJECT THE NULL HYPOTHESIS. That Means the probability of those sample, taken by chance, have a incredible small probability of be part of the same population. So we accept the NULL-HYPOTHESIS, where there are actually a true difference between the sample means of the galaxies size. 

  - With this we can say: We have two different populations of galaxies divided by their sudes and related with their color. Blue galaxxies size are different from Red galaxies size. If you want to know each is bigger, you can do a one tailed t-test.

In [ ]:
#comparing with t-test independent of Scipy
#and it seans that the t-statistic is pretty similar to our function
from scipy.stats import ttest_ind
stat, p = ttest_ind(red_sample, blue_sample)
print(f'T-statistic: {stat}')
print(f'P-value: {p}')

###Evaluating R square: 

The R square coefficient tells us how much of the variation of dependent variable can be explaned by the groups that we choosed. You can think it as: how much of he size of a galaxy can be explaned by the group of it belongs ? If a galaxy is the biggest galaxy you ever seem, and it is in the Label 0 group, how much of it's size can be explaned by this galaxy being in the label 0?

In [ ]:
r2 = (pow(t_stats,2))/((pow(t_stats,2))+df)
print(r2)

Here we got a R2 value of 0.10. That means, if we take a galaxy and look to the it's size, we could say only 10% of that could be explaned by the galaxy be part of Label 0 or Label 1. The other 90% is due to other unkow ariables, maybe like distance from Earth, the age of the galaxy, the format of the galaxy, and many other factors.

###Confidence Interval for 95%

The confidence interval teels us we 95% of our data will be in the distribution. That's great if we want to know if we have a goos estimation of some parametter. Let's see that

- IC 95% = (Mean difference +/- t_critic * Standard Error)

In [ ]:
def ic_95():
  bsm = blue_sample.mean() #blue sample mean
  rsm = red_sample.mean() #red sample mean
  dsm = rsm - bsm #difference of sample means 

  #now we need to find the square root of the differences between the menas
  #find the standard deviation of the samples. I'll be using Benso's correction fot samples
  std_rs = stdev( red_sample) #for population use pstdev()
  std_bs = stdev(blue_sample)

  #find number of samples in each group
  n_rs = len(red_sample)
  n_bs = len(blue_sample)

  #calculate standard error of the difference between the means
  dbm = sqrt(((std_rs**2)/n_rs) + ((std_bs**2)/n_bs))

  under_bounder = (dsm - cv*dbm )
  upper_bounder = (dsm + cv*dbm )

  return under_bounder, upper_bounder

In [ ]:
under, upper = ic_95()

print(f'IC95%: ({under}, {upper})')

This output means that 95% of the difference between the means are around 0.22 and 0.26. Knowing that we can say our data are far from 0, that is the point where there is no difference between the sizes of the galaxy. So we are pretty sure that this two groups have a real difference between their sizes.

## EVALUATING DISTANCE

RedShift (Mcz- distance)

Know we want to understand how the distance of the galaxies are distributed. There is more galaxies close or far from us ? These galaxies are grouped in clusters or they have an homogeneous distribution ? Let's taht a look on them:

###Evaluating the normality of the data

In [ ]:
#it seems we have a bimodal distribution. There is a small pick of galaxies relatively close to us, and another cluster far from us
combo_cluster['Mcz'].hist(bins=30)

In [ ]:
#the shape os the data can be confirmed using this qqplot
qqplot(combo_cluster['Mcz'], line='s')
plt.show()

Let's take the galaxy's distance acording to the group we classified in our BbSCAN cluster.

In [ ]:
near_sample = combo_cluster['Mcz'].loc[combo_cluster['dbscan_label']==1]
far_sample = combo_cluster['Mcz'].loc[combo_cluster['dbscan_label']==0]

In [ ]:
#distribution of galaxies in nearest group acording to DBSCAN
near_sample.hist(bins=30)

In [ ]:
#definitely this is not a normal distribution
qqplot(near_sample, line='s')
plt.show()

In [ ]:
#the far galaxies do not follow the normal distribution two
far_sample.hist(bins=30)

In [ ]:
qqplot(far_sample, line='s')
plt.show()

###Non-parametric Data

But how we can compare and how if we can truly considere this galaxies part of different groups or just an misleading of our eyes?

When we do not have a parametric data (normal distributed data) we can use non-parametric tests. There is a long debate about the statistical power of those tests, and when to apply them. Here er have two samples that do not follow the normal distribution, so we cannot compare them using standard deviation and parametric tools. But, we can compare each data is bigger than other (we cannot tell how much each point is bigger or smaler than the other, thi means we lose the magnitude of the difference between them. However, we can certainly tekll each of then is bigger tahn the other). In our casa, where we have a good number of observations (a lot of galaxies), and we want to compare two independent samples, we are able to use the Mann-Whitney non-parametric test.

mann-Withney tells us how much the two groups are separed for each other using the position of each data in a Rank of all the galaxies in our dataset, and not the absolute distance of each galaxy, just like the example before with the t-test. Here we have the steps to find the U-statistics (Mann_Whtney score).

### HYPOTHESIS TEST AND MANN-WHITNEY TEST:
  
  - H0: no difference between the medias of the distance

  - H1: there is a difference between the medians of the distances 

STEP 1: PLACE SAMPLE IN TWO COLUMNS (already done, near_sample, far_sample)

STEP 2: STACK THE THE SAMPLE IN ONE COLUMN RANKED BY THE VALUE (the smaller gets one and the bgger gets (n)

STEP 3: FOR TIED RANKS, SUM THE **POSITION** OF ALL THE THIS TIED DATAS AND GET THEIR AVERAGE.

STEP 4: ASIGN EACH POSITION TO EACH NUMBER THAT YOU RANKED. NOW, INSIDE OF EACH GROUP YOU HAVE THE ABSOLUTE VALUE OF THE GALAXY DISTANCE AND IT'S POSITION N THE RANK.

STEP 5: SUM THE POSITIONS IN THE RANK INSIDE EACH GROUP. IN THE AND YOU WILL HAVE THE SUM OF RANKS FOR NEAR_SAMPLE AND THE SUM OF THE RANKS FOR FAR_SAMPLE

STEP 6: CALCULATE THE U-STATS FOR EACH GROUP USING THE FORMULA:
    
  - U-stats = SUM of the ranks - n(n+1)/2
    
  - Where n is the number of observations inside the sample

STEP 7: CHOOSE THE SMALLEST VALUE AND COMPARE IT TO THE U-CRITICAL FOR YOUR ALPHA LEVEL (0.05 IN THIS CASE). COMPARE the P-VALUE, FOUNDED WITH THE U-CRITICAL WITH YOUR ALPHA LEVEL (0.05)

STEP 8: GET YOUR RESULT


### THE MANN-WHITNEY TEST

In [ ]:
stats, p = mannwhitneyu(near_sample, far_sample)

In [ ]:
print(stats)
print(f'U-stats: {p}')

With so small P-value we can reject the null-ypothesis, where the assumption is that the distribution of the both groups are identical and there is a 50% probability that an observation from a value ramdonly selected from one population exceeds an observation ramdonly selected from the other population.

This means we are almost sure that this two samples come from a different populatoins of distance from our galaxy.

###BONUS: OUR GALAXY

As you may know, we live in the MILKWAY galaxy. our galaxy is a beautiful blue and barred spiral galaxy. But what if our galaxy could be part of the blue galaxies of teses dataset? Where the MILKWAY would far in the distribution ? Are we more or less brighter than the other galaxies in thee group?

To be able to do that we can just supose the normality of our data, as we did before (using the histogram and the qqplot) and than find the Z-score, wich is how far a value (in this case the MilkWy's Magnitude) is from the mean of the distribution in terms of standard deviation. 

- The Rmag of the Milkway its something around 21, so let's use this value.

In [ ]:
#separating our data (The magntude of the galaxies but only the ones from the Label 1 group)
blue = combo_cluster['Rmag'].loc[combo_cluster['dbscan_label']==1]

In [ ]:
#just to know:
blue.mean()

###Evaluating the normality of the data

In [ ]:
#this kind of seem a normal distributions, with soome distortions and a little bit right skewed.
blue.hist(bins=20)

In [ ]:
#the qqplot confirms what we thought before with the histogram
qqplot(blue, line='s')
plt.show()

###The Z-Score

In [ ]:
#Getting Z-score variables
x = 21 #milkway Rmag
mu = blue.mean() #mean of the distribution
std = stdev(blue) #standard deviation of the sample

In [ ]:
# finding the Z-score
z_stats = (x - mu) / std #z-score formula
print(f'Z-Score: {z_stats}')

Here we have that our galaxy is 1.36 standard deviations units away from the mean, to the left because of the  negative value. But i really want to know how much our galaxy is brighter than the other. And how we have a normal dat we also can have a Probability Density Function, and knowing the Z-score the can just calculate the integral of this value and find the probability of that the value of our galaxy or low by chance inthis sample. Thats also means that the point where our galaxy falls in the distribution tells us how much brighter is the milkway relative to other galaxies in this sample. 

In [ ]:
#p-value for left tail 
p_value = scipy.stats.norm.sf(abs(z_stats))
print(f'P-value: {p_value}')

So we now know that our galaxy is in the 8.5% brightest gaaxie in this sample. We are preaaty bright and blue, congrats Milkway!


#THE END

Thank you for visit this notebook. Here we travel across the galaxies using some mathematical and statistical tools, and also a litte bit of machine learning. I hope you have liked it! thank you a lot!!
